In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import load_model

In [2]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils 

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [3]:
#list kata yang akan dideteksi
actions = np.array(['halo','nama','saya','siapa','kamu','I','J']) # 'translate_realtimeV2.h5'

In [4]:
model = load_model('translate_realtimeV2.h5')

In [5]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.3) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

        height, width, channels = image.shape
        text_size, _ = cv2.getTextSize(' '.join(sentence), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        text_width = text_size[0]
        x = (width - text_width) // 2
        cv2.putText(image, ' '.join(sentence), (x, height - 40), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.py

<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solu